In [1]:
"""
#Loads a CSV DwC occurrence file that has been augmented with BELS locality strings
"""

'\n#Loads a CSV DwC occurrence file that has been augmented with BELS locality strings\n'

In [1]:
import pandas as pd
import matplotlib

# switch to suppress CSV writing (which is slow)
write_csv = True

In [3]:
#df_occ = pd.read_csv('TORCH-Denton-9f30f07b-e694-4f79-83c3-54fafaa98727.csv', low_memory=False)
#df_occ = pd.read_csv('r_dale_thomas_texas_bels.csv', low_memory=False)
#df_occ = pd.read_csv('r-dale-thomas-all-seinet-texas-bels.csv', low_memory=False)
#df_occ = pd.read_csv('torch_bels_locs.csv', low_memory=False, sep='\t')
# BRIT UT test
df_occ = pd.read_csv('torch_bels_locs.tsv', low_memory=False, sep='\t')

In [3]:
df_occ.shape

(1144047, 119)

In [5]:
#df_occ.columns

In [4]:
# Drop irrelevant colums
# NOTE not all of these columns occur across all collections
drop_columns= ['higherClassification','kingdom','phylum','class','order','identificationReferences','identificationRemarks','taxonRemarks','identificationQualifier','typeStatus','fieldNumber','eventID','informationWithheld','dataGeneralizations','dynamicProperties','associatedSequences','associatedTaxa','reproductiveCondition','establishmentMeans','lifeStage','sex','individualCount','samplingProtocol','preparations','continent','waterBody','islandGroup','island','rights','rightsHolder','accessRights','recordID','type','license','bibliographicCitation','datasetName','fieldNotes','countryCode','nomenclaturalCode','nomenclaturalStatus','associatedMedia','higherGeography','institutionID','georeferencedDate','datasetID','occurrenceStatus','verbatimLocality','organismID','previousIdentifications','eventTime','eventRemarks','locationAccordingTo','verbatimCoordinateSystem','footprintWKT','earliestEonOrLowestEonothem','earliestEraOrLowestErathem','earliestPeriodOrLowestSystem','earliestEpochOrLowestSeries','earliestAgeOrLowestStage','group','formation','member','identificationVerificationStatus','scientificNameID']
df_occ = df_occ.drop(columns=drop_columns, errors='ignore')

In [5]:
df_occ.shape

(1144047, 62)

In [6]:
# Find duplicate records

#bels_matchwithcoords
df_matches = df_occ[df_occ['bels_location_string'].duplicated(keep=False)]
#df_matches = df_denton[df_denton.bels_matchwithcoords

In [7]:
df_matches.shape

(804643, 62)

In [8]:
#list(df_matches.columns)

In [9]:
# Add location ID
# first create a copy to avoid copy of a slice of a DF
df_matches = df_matches.copy()
# https://stackoverflow.com/a/51110197 or https://stackoverflow.com/a/51110205
df_matches['loc_id'] = df_matches.groupby(['bels_location_string']).ngroup()

In [10]:
# Add dup loc count column for the size of each location cluster (number of duplicate locations)
#https://stackoverflow.com/a/46768694
#TODO - better name and documentation?
# This is 
df_matches['dup_loc_count'] = df_matches.groupby(['bels_location_string']).transform('size')

In [11]:
# find groups that have at least one georeference
# from https://claude.ai/chat/f9eab37d-6cc0-459c-9d24-fd78d6152d10
#import pandas as pd

def filter_locations_with_coordinates(df):
    """
    Filter location groups based on having at least one record with coordinates.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame with location_string and coordinates columns
    
    Returns:
    pandas.DataFrame: Filtered DataFrame containing all records from location groups
                     that have at least one set of coordinates
    """
    # Find locations that have at least one non-empty coordinates value
    # This handles various types of "empty" values (None, NaN, empty string)
    locations_with_coords = df[df['decimalLatitude'].notna() & (df['decimalLatitude'] != '')]['bels_location_string'].unique()
    
    # Filter the original DataFrame to keep all records from matching locations
    filtered_df = df[df['bels_location_string'].isin(locations_with_coords)]
    
    return filtered_df


# Filter locations that have at least one set of coordinates
#result = filter_locations_with_coordinates(df)
df_matches_wgeo = filter_locations_with_coordinates(df_matches)

print("\nOriginal DataFrame:")
print(df_matches.shape)
print("\nFiltered DataFrame (locations with at least one set of coordinates):")
print(df_matches_wgeo.shape)


Original DataFrame:
(804643, 64)

Filtered DataFrame (locations with at least one set of coordinates):
(494443, 64)


In [15]:
# Save to TSV
df_matches_wgeo.to_csv('TORCH_bels_matches_w_geo.tsv', sep='\t')

In [16]:
#if write_csv:
#    df_matches.to_csv('TORCH_bels_matches_loc_id.csv')

In [14]:
# find groups that have at least one georeference
# from https://claude.ai/chat/f9eab37d-6cc0-459c-9d24-fd78d6152d10
#import pandas as pd

def filter_locations_without_coordinates(df):
    """
    Filter location groups based on having NO coordinates in any record.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame with location_string and coordinates columns
    
    Returns:
    pandas.DataFrame: Filtered DataFrame containing all records from location groups
                     that have no coordinates at all
    """
    # Group by location_string and check if ALL coordinates are empty for that group
    locations_without_coords = df.groupby('bels_location_string').agg({
        'decimalLatitude': lambda x: all(pd.isna(x) | (x == ''))
    })
    
    # Get the location strings where the condition is True
    locations_no_coords = locations_without_coords[
        locations_without_coords['decimalLatitude']
    ].index
    
    # Filter the original DataFrame to keep all records from matching locations
    filtered_df = df[df['bels_location_string'].isin(locations_no_coords)]
    
    return filtered_df

#df = pd.DataFrame(data)

# Filter locations that have at least one set of coordinates
#result = filter_locations_with_coordinates(df)
df_matches_wo_geo = filter_locations_without_coordinates(df_matches)

print("\nOriginal DataFrame:")
print(df_matches.shape)
print("\nFiltered DataFrame (locations with at least one set of coordinates):")
print(df_matches_wo_geo.shape)

# Print summary
#print("\nSummary of locations without any coordinates:")
#for loc in df_matches_wo_geo['bels_location_string'].unique():
#    print(f"- {loc}")


Original DataFrame:
(804643, 64)

Filtered DataFrame (locations with at least one set of coordinates):
(310200, 64)

Summary of locations without any coordinates:
- texasnewtonforestofftexas87nearyellowbayouca3milessouthofburkevilletx
- texashardincollectedfrombigthicketarea
- texasmatagordamadislandmarsh
- texassmithtylertx
- texassmithloperpropertyendofcr2235,offhwy110,whitehousetx
- texassmithroserudmanparktylertx
- texasgregglibertycitycommunitykilgoretx
- oklahomanorthbankoflaketexoma
- texassmithlindaletx
- texassmithhwy31eastbound10mileseastofloop323,tylertx
- texassmithsheffield'swoods
- texassmithloperyardprivateyardatendofcr2235,whitehousetx
- texasanderson1milenorthoffm2330alonghollyspringscemeteryrd
- texascherokeeuniongroverdjacksonvilletx
- texassmithhwy14n4milesfromloop323,tylertx
- texassmithres402seagle
- texassmithloperyardendofcr2235,whitehousetx
- texasandersonenglingwildlifemanagementarea
- texassmithbullardturnoffonhwy69.
- texascherokeetabernaclebaptistchurchjac

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [17]:
# Save to TSV
df_matches_wo_geo.to_csv('TORCH_bels_matches_wo_geo.tsv', sep='\t')

In [15]:
# find location group records lacking geo
#bels_decimallatitude
df_nogeo = df_matches[df_matches['decimalLatitude'].isna()]
#TODO add loc ID - https://stackoverflow.com/a/51110205


In [18]:
# all dups with and without geocoords
dup_loc_count = df_matches.pivot_table(index = ['bels_location_string'], aggfunc ='size')

In [19]:
dup_loc_count.shape

(124587,)

In [20]:
if write_csv:
    dup_loc_count.to_csv('TORCH_dup_loc_count.csv')

In [21]:
# dups with no goecoords
#dups = df.pivot_table(index = ['Course'], aggfunc ='size') 
#dup_loc_count_no_geo = df_nogeo.pivot_table(index = ['bels_location_string'], aggfunc ='size')
#df_matches_wo_geo
dup_loc_count_no_geo = df_matches_wo_geo.pivot_table(index = ['bels_location_string'], aggfunc ='size')

In [25]:
dup_loc_count_no_geo.shape

(65165,)

In [24]:
if write_csv:
    dup_loc_count_no_geo.to_csv('TORCH_dup_loc_count_wo_geo.tsv', sep='\t')

In [26]:
# dups with goecoords
#dup_loc_count_geo = df_geo.pivot_table(index = ['bels_location_string'], aggfunc ='size')
dup_loc_count_geo = df_matches_wgeo.pivot_table(index = ['bels_location_string'], aggfunc ='size')

In [27]:
dup_loc_count_geo.shape

(59422,)

In [28]:
if write_csv:
    dup_loc_count_geo.to_csv('TORCH_dup_loc_count_w_geo.tsv', sep='\t')

In [140]:
#print(dup_loc_count)

In [141]:
#filter records that mention no additional locality?

In [142]:
#list(df_matches.columns)

In [143]:
# from ChatGPT
#https://chat.openai.com/share/21c561bb-db43-47ee-980a-f88078b8c78b
# Identify duplicate values in a specific column
#
#duplicates = df_matches['bels_matchwithcoords'].duplicated(keep=False)
#duplicates = df_matches['bels_location_string'].duplicated(keep=False)

# Count the number of duplicates for each value
#duplicate_counts = duplicates.groupby(df_matches['bels_matchwithcoords']).sum()
#duplicate_counts = duplicates.groupby(df_matches['bels_location_string']).sum()



In [144]:
# Note needed, done above
# Add a new column to the DataFrame with the number of duplicates
#df_matches['duplicate_count'] = df_matches['bels_location_string'].map(duplicate_counts)

In [145]:
#duplicate_counts.sort_index().cumsum().plot()
#duplicate_counts.set_index('duplicate_count')
#duplicate_counts.sort_index().plot()

In [146]:
#df_matches[['duplicate_count']] = df_matches[['duplicate_count']].apply(pd.to_numeric)

In [147]:
#df_matches.set_index('duplicate_count').cumsum()

In [148]:
#TODO
#Load ids of elite georeferencers - EG
df_georeferencers = pd.read_csv('TORCH-georeferencers_tested.csv', low_memory=False)

#find records with geo that have been done by EGs
#find records without geo that match those done by EGs

In [149]:
# filter records that have been georeferenced by 'elite' vetted georeferencers
df_by_egeo = df_matches[df_matches['georeferencedBy'].isin(df_georeferencers['Username'])]

In [150]:
df_by_egeo.shape

(30372, 64)

In [151]:
if write_csv:
    df_by_egeo.to_csv('TORCH_df_by_egeo.csv')

In [152]:
# find locations that match loc_id of those done by EG
df_loc_match_by_egeo = df_matches[df_matches['loc_id'].isin(df_by_egeo['loc_id'])]

In [153]:
df_loc_match_by_egeo.shape

(45211, 64)

In [154]:
# find matching locs that have not been georeffed
df_loc_match_by_egeo_nogeo = df_loc_match_by_egeo[df_loc_match_by_egeo['decimalLatitude'].isna()]

In [155]:
df_loc_match_by_egeo_nogeo.shape

(9424, 64)

In [156]:
# find matching locs that have been georeffed
df_loc_match_by_egeo_w_geo = df_loc_match_by_egeo[df_loc_match_by_egeo['decimalLatitude'].notna()]

In [157]:
df_loc_match_by_egeo_w_geo.shape

(35787, 64)

In [158]:
df_nogeo.shape

(208011, 64)

In [30]:
# find dup locs without geo but have other dups that have geo
df_nogeo_with_geolocdups = df_nogeo[df_nogeo['loc_id'].isin(df_geo['loc_id'])]
#df_matches_wo_geo
#df_nogeo_with_geolocdups = df_matches_wo_geo[df_matches_wo_geo['loc_id'].isin(df_geo['loc_id'])]


NameError: name 'df_nogeo' is not defined

In [160]:
df_nogeo_with_geolocdups.shape

(54747, 64)

In [161]:
if write_csv:
    df_nogeo_with_geolocdups.to_csv('TORCH_nogeo_with_geolocdups.csv')

In [162]:
#TODO 



In [31]:
# Drop records with only the state name in bels string
#df_nogeo_with_geolocdups_nostates = df_nogeo_with_geolocdups[df_nogeo_with_geolocdups[column1] != df[column2]]

#df_nogeo_with_geolocdups_nostates = df_nogeo_with_geolocdups[~df_nogeo_with_geolocdups.apply(lambda row: row['stateProvince'].lower() == row['bels_location_string'].lower(), axis=1)]
#df_matches_wo_geo
df_wo_geo_nostates = df_matches_wo_geo[~df_matches_wo_geo.apply(lambda row: row['stateProvince'].lower() == row['bels_location_string'].lower(), axis=1)]


In [32]:
#df_nogeo_with_geolocdups_nostates.shape
df_wo_geo_nostates.shape

(309918, 64)

In [165]:
if write_csv:
    df_nogeo_with_geolocdups_nostates.to_csv('TORCH_nogeo_with_geolocdups_nostates.csv')

In [33]:
# remove state + county matches
#df_nogeo_with_geolocdups_nostatecounty = df_nogeo_with_geolocdups_nostates[~df_nogeo_with_geolocdups_nostates.apply(lambda row: (str(row['stateProvince']) + str(row['county'])).lower() == row['bels_location_string'].lower(), axis=1)]
#df_wo_geo_nostates
df_wo_geo_nostatecounty = df_wo_geo_nostates[~df_wo_geo_nostates.apply(lambda row: (str(row['stateProvince']) + str(row['county'])).lower() == row['bels_location_string'].lower(), axis=1)]


In [34]:
#df_nogeo_with_geolocdups_nostatecounty.shape
df_wo_geo_nostatecounty.shape

(277948, 64)

In [168]:
if write_csv:
    df_nogeo_with_geolocdups_nostatecounty.to_csv('TORCH_nogeo_with_geolocdups_nostatecounties.csv')

In [169]:
list[df_nogeo_with_geolocdups_nostatecounty.columns]

list[Index(['id', 'institutionCode', 'collectionCode', 'ownerInstitutionCode',
       'collectionID', 'basisOfRecord', 'occurrenceID', 'catalogNumber',
       'otherCatalogNumbers', 'family', 'scientificName', 'taxonID',
       'scientificNameAuthorship', 'genus', 'subgenus', 'specificEpithet',
       'verbatimTaxonRank', 'infraspecificEpithet', 'taxonRank',
       'identifiedBy', 'dateIdentified', 'recordedBy', 'recordNumber',
       'eventDate', 'year', 'month', 'day', 'startDayOfYear', 'endDayOfYear',
       'verbatimEventDate', 'occurrenceRemarks', 'habitat',
       'associatedOccurrences', 'locationID', 'country', 'stateProvince',
       'county', 'municipality', 'locality', 'locationRemarks',
       'decimalLatitude', 'decimalLongitude', 'geodeticDatum',
       'coordinateUncertaintyInMeters', 'verbatimCoordinates',
       'georeferencedBy', 'georeferenceProtocol', 'georeferenceSources',
       'georeferenceVerificationStatus', 'georeferenceRemarks',
       'minimumElevationInMet

In [35]:
# add counts by county
#df_nogeo_with_geolocdups_nostatecounty
#df_wo_geo_nostatecounty
county_summary = df_wo_geo_nostatecounty.groupby(['stateProvince','county']).agg(
    total_locations=('loc_id', 'count'),
    #sum_id_count=('dup_loc_count', 'sum'),
    unique_locations=('loc_id', 'nunique'),
    dup_loc_count=('dup_loc_count', 'max')
)

In [36]:
county_summary

total_locations  unique_locations  dup_loc_count
stateProvince county                                                      
Oklahoma      Adair                  1381               295             57
              Alfalfa                 265                72             18
              Angelina                  2                 1              2
              Atoka                  1227               254             53
              Atoka?                    1                 1            107
...                                   ...               ...            ...
Texas         not listed              807                53            316
              presidio                  1                 1              4
              starr                     1                 1              2
              walker                    2                 2              2
              willacy                   1                 1              2

[477 rows x 3 columns]

In [38]:
county_summary.to_csv('TORCH_wo_geo_county_summary.tsv', sep='\t')

In [173]:
# returning to unfiltered DF for more comprehensive results

#df_nogeo
# remove states only
df_nogeo_nostates = df_nogeo[~df_nogeo.apply(lambda row: row['stateProvince'].lower() == row['bels_location_string'].lower(), axis=1)]
#remove state and county only
df_nogeo_nostatecounty = df_nogeo_nostates[~df_nogeo_nostates.apply(lambda row: (str(row['stateProvince']) + str(row['county'])).lower() == row['bels_location_string'].lower(), axis=1)]

df_nogeo_nostatecounty.shape


(202546, 64)

In [174]:
county_summary_nogeo = df_nogeo_nostatecounty.groupby(['stateProvince','county']).agg(
    #total_locations=('loc_id', 'count'),
    unique_locations=('loc_id', 'nunique'),
    #sum_id_count=('dup_loc_count', 'sum')
    loc_count=('loc_id', 'count')
)
# Reset the index to make 'state' and 'county' regular columns
county_summary_nogeo = county_summary_nogeo.reset_index()
print("\nSummary with reset index:")
print(county_summary_nogeo)


Summary with reset index:
    stateProvince    county  unique_locations  loc_count
0        Oklahoma     Adair               113        433
1        Oklahoma   Alfalfa                29         80
2        Oklahoma  Angelina                 1          2
3        Oklahoma     Atoka               168        857
4        Oklahoma    Beaver                30        147
..            ...       ...               ...        ...
345         Texas    Yoakum                 7         23
346         Texas     Young                50        151
347         Texas    Zapata               161        439
348         Texas    Zavala                47        165
349         Texas  mitchell                 1          1

[350 rows x 4 columns]


In [175]:
# Same process but only include BRIT records in Texas
# returning to unfiltered DF for more comprehensive results

#df_nogeo
# Texas collections only
df_torch_texas_nogeo = df_nogeo[df_nogeo['stateProvince'] == 'Texas']
# remove states only
df_torch_texas_nogeo_nostates = df_torch_texas_nogeo[~df_torch_texas_nogeo.apply(lambda row: row['stateProvince'].lower() == row['bels_location_string'].lower(), axis=1)]
#remove state and county only
df_torch_texas_nogeo_nostatecounty = df_torch_texas_nogeo_nostates[~df_torch_texas_nogeo_nostates.apply(lambda row: (str(row['stateProvince']) + str(row['county'])).lower() == row['bels_location_string'].lower(), axis=1)]


In [176]:
df_torch_texas_nogeo_nostatecounty.shape  # old (106414, 64)

(175474, 64)

In [177]:
# BRIT collections only
df_torch_texas_brit_nogeo = df_torch_texas_nogeo_nostatecounty[(df_torch_texas_nogeo_nostatecounty['institutionCode'] == 'BRIT') | (df_torch_texas_nogeo_nostatecounty['institutionCode'] == 'VDB')]

df_torch_texas_brit_nogeo.shape


(72524, 64)

In [178]:
df_torch_texas_brit_nogeo.to_csv('torch_texas_brit_nogeo.csv')

In [179]:
# Summarize BRIT Texas records
brit_texas_county_summary_nogeo = df_torch_texas_brit_nogeo.groupby(['stateProvince','county']).agg(
    #total_locations=('loc_id', 'count'),
    unique_locations=('loc_id', 'nunique'),
    #sum_id_count=('dup_loc_count', 'sum')
    loc_count=('loc_id', 'count'),
    largest_dups = ('dup_loc_count', 'max')
)
# Reset the index to make 'state' and 'county' regular columns
brit_texas_county_summary_nogeo = brit_texas_county_summary_nogeo.reset_index()
print("\nSummary with reset index:")
print(brit_texas_county_summary_nogeo)


Summary with reset index:
    stateProvince    county  unique_locations  loc_count  largest_dups
0           Texas  Anderson               204        809            77
1           Texas   Andrews                 3          4             4
2           Texas  Angelina               136        421            70
3           Texas   Aransas               175        725           249
4           Texas    Archer                38        145            44
..            ...       ...               ...        ...           ...
225         Texas      Wise                79        221            68
226         Texas      Wood               185        882            58
227         Texas     Young                35         99            47
228         Texas    Zapata               133        337            65
229         Texas    Zavala                28         85            16

[230 rows x 5 columns]


In [180]:
#df_torch_texas_nogeo_nostatecounty.shape
# export BRIT Texas summary
brit_texas_county_summary_nogeo.to_csv('TORCH_brit_texas_county_summary_nogeo.csv')

In [181]:
#county_summary_nogeo.to_csv('TORCH_county_summary_nogeo.csv')

In [182]:
# Sampling individual counties
#df_torch_texas_brit_nogeo
# df_torch_texas_brit_nogeo = df_torch_texas_nogeo_nostatecounty[(df_torch_texas_nogeo_nostatecounty['institutionCode'] == 'BRIT') | (df_torch_texas_nogeo_nostatecounty['institutionCode'] == 'VDB')]

df_brit_young_nogeo = df_torch_texas_brit_nogeo[df_torch_texas_brit_nogeo['county'] == 'Young']

df_brit_young_nogeo.shape



(99, 64)

In [183]:
df_brit_young_nogeo.to_csv('brit_young_nogeo.csv')